In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
import plotly.express as px
from tensorflow.keras.models import load_model
import warnings
import plotly.figure_factory as ff
import plotly.express as px
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os
from sklearn.model_selection import train_test_split
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder
warnings.simplefilter(action='ignore')

In [4]:
filename=r"D:\testekeras\notebooks\data_superstore.csv"

In [5]:
dados = pd.read_csv(filename,sep=";",encoding="utf-8",parse_dates=['Data do pedido', 'Data de envio'])

In [6]:
dados[['Vendas','Lucro','Desconto']] = dados[['Vendas','Lucro','Desconto']].applymap(lambda k: float(str(k).replace(",",".")))
dados['Quantidade'] = dados['Quantidade'].astype(int)

In [7]:
dados.columns

Index(['ID da fila', 'ID do pedido', 'Devolvido', 'Data do pedido',
       'Data de envio', 'Modo de envio', 'ID do cliente', 'Nome do cliente',
       'Segmento', 'Cidade', 'Estado', 'País/Região', 'Região',
       'ID do produto', 'Categoria', 'Sub-categoria', 'Nome do produto',
       'Vendas', 'Quantidade', 'Desconto', 'Lucro'],
      dtype='object')

In [8]:
dados

,ID da fila,ID do pedido,Devolvido,Data do pedido,Data de envio,Modo de envio,ID do cliente,Nome do cliente,Segmento,Cidade,...,País/Região,Região,ID do produto,Categoria,Sub-categoria,Nome do produto,Vendas,Quantidade,Desconto,Lucro
0,1,MX-2021-143658,Não,02/10/2021,06/10/2021,Classe padrão,SR-20575,Sofia Rocha,Varejo,Cidade do México,...,México,Norte,MAT-ET-10001014,Material de escritório,Etiquetas,"HON Etiquetas para organização de pastas, Ajus...",130.80,3,0.0,45.60
1,2,MX-2019-155047,Não,16/10/2019,21/10/2019,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,...,Colômbia,Sul,MÓV-MÓ-10004592,Móveis,Móveis,"Tenex Relógio de parede, Durável",2521.60,8,0.0,907.20
2,3,MX-2019-155047,Não,16/10/2019,21/10/2019,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,...,Colômbia,Sul,MÓV-ES-10000849,Móveis,Estantes,"Ikea Gaveteiro, Branco",1932.80,2,0.0,540.80
3,4,MX-2019-155047,Não,16/10/2019,21/10/2019,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,...,Colômbia,Sul,MAT-FI-10003284,Material de escritório,Fichários,"Cardinal Pasta argolada, Transparente",354.40,4,0.0,49.60
4,5,MX-2019-155047,Não,16/10/2019,21/10/2019,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,...,Colômbia,Sul,MAT-AR-10002762,Material de escritório,Arte,"Sanford Tela, Tamanhos variados",716.00,2,0.0,114.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,10283,US-2019-145282,Não,05/04/2019,08/04/2019,Primeira classe,LM-17230,Letícia Martins,Varejo,Ponte Nova,...,Brasil,Sul,MAT-ET-10003909,Material de escritório,Etiquetas,"HON Etiquetas de envio, Ajustável",74.16,3,0.6,-81.84
10258,10284,US-2019-145282,Não,05/04/2019,08/04/2019,Primeira classe,LM-17230,Letícia Martins,Varejo,Ponte Nova,...,Brasil,Sul,MAT-EN-10001954,Material de escritório,Envelopes,"Jiffy Fita dupla face, Vermelho",62.80,1,0.6,-64.40
10259,10285,US-2019-106523,Não,11/02/2019,15/02/2019,Classe padrão,RA-19795,Raissa Azevedo,Pequenas e Medias Empresas,São Miguel dos Campos,...,Brasil,Sul,MÓV-CA-10004481,Móveis,Cadeiras,"Harbour Creations Cadeira de praia, Vermelho",492.96,3,0.6,-407.04
10260,10286,US-2020-168032,Não,05/11/2020,09/11/2020,Classe padrão,BC-11380,Brenda Cardoso,Grande Empresa,Vassouras,...,Brasil,Sul,MÓV-ES-10003288,Móveis,Estantes,"Ikea Par, Preto",663.52,2,0.6,-199.28


In [9]:
dados.fillna(0, inplace=True)

In [10]:

colunas_para_agrupar = ["ID do cliente", "Cidade", "Categoria", "Modo de envio"]


i = 0


while i < len(colunas_para_agrupar):
    coluna = colunas_para_agrupar[i]
    nome_coluna_nova = f'QuantidadePedido{coluna.replace(" ", "")}'
    
    dados_filtrados = dados[dados['Devolvido'] == 'Sim']
    
    
    dados_agrupados = dados_filtrados.groupby(coluna)['ID do pedido'].nunique().reset_index()
    dados_agrupados.rename(columns={'ID do pedido': nome_coluna_nova}, inplace=True)
    
    dados = pd.merge(dados, dados_agrupados, on=coluna, how='left')
    
    dados[nome_coluna_nova].fillna(0, inplace=True)
    
    i += 1

print(dados.head(3),dados.columns,dados.dtypes)


   ID da fila    ID do pedido Devolvido Data do pedido Data de envio  \
0           1  MX-2021-143658       Não     02/10/2021    06/10/2021   
1           2  MX-2019-155047       Não     16/10/2019    21/10/2019   
2           3  MX-2019-155047       Não     16/10/2019    21/10/2019   

   Modo de envio ID do cliente  Nome do cliente Segmento            Cidade  \
0  Classe padrão      SR-20575      Sofia Rocha   Varejo  Cidade do México   
1  Classe padrão      LC-16570  Larissa Cardoso   Varejo     Dos Quebradas   
2  Classe padrão      LC-16570  Larissa Cardoso   Varejo     Dos Quebradas   

   ... Sub-categoria                                    Nome do produto  \
0  ...     Etiquetas  HON Etiquetas para organização de pastas, Ajus...   
1  ...        Móveis                   Tenex Relógio de parede, Durável   
2  ...      Estantes                             Ikea Gaveteiro, Branco   

   Vendas Quantidade Desconto  Lucro QuantidadePedidoIDdocliente  \
0   130.8          3      0.0

In [11]:
def preprocess_categorical_columns(data):
    categorical_columns = data.select_dtypes(include='object').columns
    
    categorical_data = data[categorical_columns]
    
    encoder = OneHotEncoder(sparse=False, drop='first')
    
    encoded_data = encoder.fit_transform(categorical_data)
    
 
    new_column_names = encoder.get_feature_names_out(input_features=categorical_columns)
    

    encoded_df = pd.DataFrame(encoded_data, columns=new_column_names, index=data.index)
    
   
    data = data.drop(categorical_columns, axis=1)
    
    data = pd.concat([data, encoded_df], axis=1)
    
    return data



In [12]:
pd.options.display.float_format = '{:.2f}'.format


In [13]:
dados = preprocess_categorical_columns(dados)
dados

,ID da fila,Vendas,Quantidade,Desconto,Lucro,QuantidadePedidoIDdocliente,QuantidadePedidoCidade,QuantidadePedidoCategoria,QuantidadePedidoMododeenvio,ID do pedido_MX-2018-100174,...,"Nome do produto_Xerox Caderno, Premium","Nome do produto_Xerox Caderno, Reciclado","Nome do produto_Xerox Papel de pergaminho, Embalagem com 12","Nome do produto_Xerox Papel de pergaminho, Multicolorido","Nome do produto_Xerox Papel de pergaminho, Premium","Nome do produto_Xerox Papel de pergaminho, Reciclado","Nome do produto_Xerox Resma, Embalagem com 12","Nome do produto_Xerox Resma, Multicolorido","Nome do produto_Xerox Resma, Premium","Nome do produto_Xerox Resma, Reciclado"
0,1,130.80,3,0.00,45.60,1.00,20.00,244,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2,2521.60,8,0.00,907.20,1.00,0.00,130,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,3,1932.80,2,0.00,540.80,1.00,0.00,130,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,4,354.40,4,0.00,49.60,1.00,0.00,244,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,5,716.00,2,0.00,114.40,1.00,0.00,244,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,10283,74.16,3,0.60,-81.84,0.00,0.00,244,40,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10258,10284,62.80,1,0.60,-64.40,0.00,0.00,244,40,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10259,10285,492.96,3,0.60,-407.04,0.00,0.00,130,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10260,10286,663.52,2,0.60,-199.28,0.00,0.00,130,182,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [14]:


corr_matrix = dados.corr()


title = 'Matriz de Correlação'


fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=corr_matrix.round(2).values,
    colorscale='Viridis', 
)


fig.update_layout(
    title=title,
    xaxis=dict(ticks='', side='top'),
    yaxis=dict(ticks='', ticksuffix=' ', tickfont=dict(size=10)),
)


fig.show()


In [ ]:



colors = {
    '1': 'blue',
    '2': 'red',
    '3': 'green',
    '4': 'purple'
}


custom_template = {
    'layout': {
        'paper_bgcolor': 'black', 
        'plot_bgcolor': 'black',  
    },
    'data': {
        'scatter': {
            'marker': {
                'colorscale': [[0, colors['1']], [0.33, colors['2']], [0.67, colors['3']], [1, colors['4']]],
            }
        },
    }
}

fig = px.scatter_matrix(
    dados,
    dimensions=['dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
       'weathersit', 'temp', 'hum', 'windspeed', 'casual', 'registered',
       'cnt'],
    color='season',
    title="Pairplot using Plotly",
    template='plotly_dark', 
)

fig.update_traces(diagonal_visible=False)


fig.update_layout(template=custom_template)


fig.update_layout(
    autosize=False,
    width=1200,  
    height=1200 
)


fig.update_xaxes(tickangle=90)

fig.show()


In [ ]:
dados.columns

Index(['ID da fila', 'Vendas', 'Quantidade', 'Desconto', 'Lucro',
       'QuantidadePedidoIDdocliente', 'QuantidadePedidoCidade',
       'QuantidadePedidoCategoria', 'QuantidadePedidoMododeenvio',
       'ID do pedido_MX-2018-100174',
       ...
       'Nome do produto_Xerox Caderno, Premium',
       'Nome do produto_Xerox Caderno, Reciclado',
       'Nome do produto_Xerox Papel de pergaminho, Embalagem com 12',
       'Nome do produto_Xerox Papel de pergaminho, Multicolorido',
       'Nome do produto_Xerox Papel de pergaminho, Premium',
       'Nome do produto_Xerox Papel de pergaminho, Reciclado',
       'Nome do produto_Xerox Resma, Embalagem com 12',
       'Nome do produto_Xerox Resma, Multicolorido',
       'Nome do produto_Xerox Resma, Premium',
       'Nome do produto_Xerox Resma, Reciclado'],
      dtype='object', length=14034)

In [ ]:
X = dados.drop('Vendas', axis=1)
y = dados['Vendas']


In [ ]:

y = np.array(y)


y = y.reshape(-1, 1)


scaler = MinMaxScaler()
y = scaler.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
y

array([[0.00331575],
       [0.07216501],
       [0.05520899],
       ...,
       [0.01374508],
       [0.0186568 ],
       [0.0789382 ]])

In [ ]:
num_features = X_train.shape[1] 

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=100, activation='relu', input_shape=(num_features,)))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='linear'))

model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 100)               1403400   
                                                                 
 dense_19 (Dense)            (None, 100)               10100     
                                                                 
 dense_20 (Dense)            (None, 100)               10100     
                                                                 
 dense_21 (Dense)            (None, 1)                 101       
                                                                 
Total params: 1423701 (5.43 MB)
Trainable params: 1423701 (5.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='Adam', loss='mean_squared_error')

In [ ]:
epochs_hist = model.fit(X_train, y_train, epochs = 25, batch_size = 50, validation_split=0.2)

Epoch 1/25
132/132 [==============================] - 3s 20ms/step - loss: 135.3681 - val_loss: 0.0999
Epoch 2/25
132/132 [==============================] - 2s 16ms/step - loss: 0.0878 - val_loss: 0.0645
Epoch 3/25
132/132 [==============================] - 2s 16ms/step - loss: 0.3607 - val_loss: 0.0702
Epoch 4/25
132/132 [==============================] - 2s 16ms/step - loss: 0.0408 - val_loss: 0.0318
Epoch 5/25
132/132 [==============================] - 2s 16ms/step - loss: 0.0228 - val_loss: 0.0286
Epoch 6/25
132/132 [==============================] - 2s 16ms/step - loss: 0.0595 - val_loss: 0.2009
Epoch 7/25
132/132 [==============================] - 2s 17ms/step - loss: 0.0725 - val_loss: 0.0214
Epoch 8/25
132/132 [==============================] - 2s 16ms/step - loss: 0.0201 - val_loss: 0.0203
Epoch 9/25
132/132 [==============================] - 2s 17ms/step - loss: 0.0234 - val_loss: 0.0261
Epoch 10/25
132/132 [==============================] - 2s 17ms/step - loss: 0.3415 - val_

In [ ]:

model.save('nome_do_modelo.h5')


In [ ]:



modelo = tf.keras.models.load_model('nome_do_modelo.h5')

previsoes = modelo.predict(X_test)



65/65 [==============================] - 0s 3ms/step


In [ ]:
y_predict_1 = scaler.inverse_transform(previsoes)


In [ ]:
y_predict_orig = scaler.inverse_transform(y_predict_1)
y_test_orig = scaler.inverse_transform(y_test)

In [ ]:
k = X_test.shape[1]
n = len(X_test)
n

2053

In [ ]:


RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_predict_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_predict_orig)
MAE = mean_absolute_error(y_test_orig, y_predict_orig)
r2 = r2_score(y_test_orig, y_predict_orig)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2, '\nAdjusted R2 =', adj_r2) 

RMSE = 0.105 
MSE = 0.011040771890397752 
MAE = 0.07514579844576862 
R2 = -0.28341382874755716 
Adjusted R2 = 1.2198118000659366


In [ ]:

df_resultados = pd.DataFrame({'Vendas Real': y_test.flatten(), 'Vendas Previstas': previsoes.flatten()})


print(df_resultados.head())


from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, previsoes)
r2 = r2_score(y_test, previsoes)

print(f'MSE (Mean Squared Error): {mse}')
print(f'R2 Score (Coefficient of Determination): {r2}')


   Vendas Real  Vendas Previstas
0         0.02             -0.07
1         0.02              0.06
2         0.01             -0.04
3         0.04              0.08
4         0.01              0.06
MSE (Mean Squared Error): 0.011040771890397752
R2 Score (Coefficient of Determination): -0.28341382874755716


In [ ]:

epochs_hist.history.keys()

dict_keys(['loss', 'val_loss'])

In [ ]:
y_predict_orig = scaler.inverse_transform(X_test)
y_test_orig = scaler.inverse_transform(y_test)

In [ ]:
y_predict_orig

array([[2.514e+03, 3.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [7.215e+03, 4.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [9.370e+02, 2.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.621e+03, 5.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [3.508e+03, 2.000e+00, 4.000e-01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [6.689e+03, 6.000e+00, 4.000e-01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])